In [1]:
import pandas as pd

In [3]:
data_movies= pd.read_csv("data/dataset.csv")

In [4]:
data_movies.head(2)

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731


In [15]:
data_movies.head(165)["genre"]

0               Drama,Crime
1      Comedy,Drama,Romance
2               Drama,Crime
3         Drama,History,War
4               Drama,Crime
               ...         
160                   Drama
161      Action,Crime,Drama
162                  Comedy
163    Drama,Thriller,Crime
164     Crime,Drama,Mystery
Name: genre, Length: 165, dtype: object

In [5]:
print("Data Movies", data_movies.shape)

Data Movies (10000, 9)


In [6]:
data_movies.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [7]:
data_movies.rename(columns = {"id":"movie_id"}, inplace = True)

In [8]:
data_movies.columns

Index(['movie_id', 'title', 'genre', 'original_language', 'overview',
       'popularity', 'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [9]:
data_movies['tags']=data_movies['overview']+data_movies['genre']

In [10]:
data_movies.shape

(10000, 10)

In [11]:
data_movies.head(2)

,movie_id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count,tags
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731,"Raj is a rich, carefree, happy-go-lucky second..."


In [12]:
newrec_data=data_movies[['movie_id', 'title', 'tags','genre','original_language','release_date','overview']]

In [13]:
newrec_data.head(3)

,movie_id,title,tags,genre,original_language,release_date,overview
0,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"Drama,Crime",en,1994-09-23,Framed in the 1940s for the double murder of h...
1,19404,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","Comedy,Drama,Romance",hi,1995-10-19,"Raj is a rich, carefree, happy-go-lucky second..."
2,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","Drama,Crime",en,1972-03-14,"Spanning the years 1945 to 1955, a chronicle o..."


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
cv=CountVectorizer(max_features=10000, stop_words='english')

In [16]:
cv

CountVectorizer(max_features=10000, stop_words='english')

In [17]:
vector=cv.fit_transform(newrec_data['tags'].values.astype('U')).toarray()

In [18]:
vector.shape

(10000, 10000)

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
similarity=cosine_similarity(vector)

In [21]:
similarity

array([[1.        , 0.05634362, 0.12888482, ..., 0.07559289, 0.11065667,
        0.06388766],
       [0.05634362, 1.        , 0.07624929, ..., 0.        , 0.03636965,
        0.        ],
       [0.12888482, 0.07624929, 1.        , ..., 0.02273314, 0.06655583,
        0.08645856],
       ...,
       [0.07559289, 0.        , 0.02273314, ..., 1.        , 0.03253   ,
        0.02817181],
       [0.11065667, 0.03636965, 0.06655583, ..., 0.03253   , 1.        ,
        0.0412393 ],
       [0.06388766, 0.        , 0.08645856, ..., 0.02817181, 0.0412393 ,
        1.        ]])

In [22]:
newrec_data[newrec_data['title']=="The Godfather"].index[0]

2

In [23]:
distance = sorted(list(enumerate(similarity[2])), reverse=True, key=lambda vector:vector[1])
for i in distance[0:5]:
    print(newrec_data.iloc[i[0]].title)

The Godfather
The Godfather: Part II
Blood Ties
Joker
Bomb City


In [24]:
def recommand(movie_name):
    index=newrec_data[newrec_data['title']==movie_name].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
    for i in distance[0:6]:
        print(newrec_data.iloc[i[0]].title)

In [25]:
recommand('Batman')

Batman
Batman Returns
The Batman vs. Dracula
Batman Beyond: Return of the Joker
The Dark Knight
Batman Begins


In [26]:
import pickle

In [27]:
pickle.dump(newrec_data, open('movie_list.pkl', 'wb'))

In [28]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))

In [29]:
V = data_movies['vote_count']
R = data_movies['vote_average']
C = data_movies['vote_average'].mean()
m = data_movies['vote_count'].quantile(0.75)

data_movies['weighted_average'] = (V/(V+m) * R) + (m/(m+V) * C)

In [30]:
movies_ranked = data_movies.sort_values('weighted_average', ascending=False)
movies_ranked[['title', 'vote_count', 'vote_average', 'weighted_average', 'popularity']].head(20)

,title,vote_count,vote_average,weighted_average,popularity
0,The Shawshank Redemption,21862,8.7,8.569860,94.075
2,The Godfather,16280,8.7,8.528911,90.585
15,The Dark Knight,27925,8.5,8.406649,82.230
3,Schindler's List,12959,8.6,8.399631,44.761
17,Pulp Fiction,23481,8.5,8.390016,67.224
20,Forrest Gump,23009,8.5,8.387894,65.175
18,The Lord of the Rings: The Return of the King,19844,8.5,8.371239,134.707
4,The Godfather: Part II,9811,8.6,8.343668,57.749
13,The Green Mile,14162,8.5,8.324407,69.118
12,Parasite,14036,8.5,8.322979,65.147


In [31]:
pickle.dump(data_movies, open('alldata.pkl', 'wb'))

In [32]:
data_genre=data_movies[["genre"]]

In [33]:
data_genre.head()["genre"]

0             Drama,Crime
1    Comedy,Drama,Romance
2             Drama,Crime
3       Drama,History,War
4             Drama,Crime
Name: genre, dtype: object

In [34]:
split_genre = data_genre['genre'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('new_genre')

final_genre = data_movies.drop('genre', axis=1).join(split_genre)

In [35]:
final_genre.head()

,movie_id,title,original_language,overview,popularity,release_date,vote_average,vote_count,tags,weighted_average,new_genre
0,278,The Shawshank Redemption,en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862,Framed in the 1940s for the double murder of h...,8.569860,Drama
0,278,The Shawshank Redemption,en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862,Framed in the 1940s for the double murder of h...,8.569860,Crime
1,19404,Dilwale Dulhania Le Jayenge,hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731,"Raj is a rich, carefree, happy-go-lucky second...",8.115311,Comedy
1,19404,Dilwale Dulhania Le Jayenge,hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731,"Raj is a rich, carefree, happy-go-lucky second...",8.115311,Drama
1,19404,Dilwale Dulhania Le Jayenge,hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731,"Raj is a rich, carefree, happy-go-lucky second...",8.115311,Romance


In [36]:
pickle.dump(final_genre, open('finalgenre.pkl','wb'))

In [37]:
final_genre.head(70)["new_genre"]

0         Drama
0         Crime
1        Comedy
1         Drama
1       Romance
        ...    
28       Horror
28        Drama
28     Thriller
29    Animation
29        Drama
Name: new_genre, Length: 70, dtype: object